In [35]:
import gpytorch
import pyreadr
import pooling_params as gtp
import run_gpytorchkernel

In [15]:
import numpy as np

In [2]:
import os

In [21]:
def fake_data(n):
    #temperature, last steps, prior,location
    return [[np.random.random()*50+10,np.random.random()*30,np.random.random()*20,int(np.random.random()>.5)] for i in range(n)]

In [22]:
data = fake_data(100)

In [49]:
def initialize_policy_params_TS(standardize=False,baseline_features=None,psi_features=None,\
                                responsivity_keys=None):
    #,'location_1','location_2','location_3'
    #'continuous_temp',
    global_p =gtp.TS_global_params(21,baseline_features=baseline_features,psi_features=psi_features, responsivity_keys= responsivity_keys)
    #personal_p = pp.TS_personal_params()
    #global_p =gtp.TS_global_params(10,context_dimension)
    
    
    
    #global_p.mu_dimension = 64
    
    global_p.kdim =24
    #194
    global_p.baseline_indices = [i for i in range(3+ 3*len(baseline_features))]
    #[i for i in range(192)]
    #[0,1,2,3,4,5,6]
    global_p.psi_indices = [0] + [1+baseline_features.index(j) for j in psi_features] \
    + [len(baseline_features)+1] + [(2+len(baseline_features))+baseline_features.index(j) for j in psi_features]
    #[0,64]
    global_p.user_id_index =0
    
    global_p.psi_features =psi_features
    #[0,64]
    
    #print(global_p.psi_indices )
    
    #global_p.update_period = update_period
    
    global_p.standardize = standardize
    global_p.lr = .01

    initial_context = [0 for i in range(global_p.theta_dim)]
    
    global_p.mus0= global_p.get_mu0(initial_context)
    #global_p.get_mu0(initial_context)
    global_p.mus1= global_p.get_mu1(global_p.num_baseline_features)
    global_p.mus2= global_p.get_mu2(global_p.num_responsivity_features)
    #np.array([.120,3.3,-.11])
    #global_p.get_mu2(global_p.num_responsivity_features)
    
    #global_p.sigmas0= global_p.get_asigma(len( personal_p.mus0[person]))
    global_p.sigmas1= global_p.get_asigma(global_p.num_baseline_features+1)
    global_p.sigmas2= global_p.get_asigma( global_p.num_responsivity_features+1)
    

    global_p.mu2_knot = np.array([0]+[0 for i in range(global_p.num_responsivity_features)])
    global_p.mu1_knot = np.zeros(global_p.num_baseline_features+1)
    global_p.sigma1_knot = np.eye(global_p.num_baseline_features+1)
    global_p.sigma2_knot = np.eye(global_p.num_responsivity_features+1)

    return global_p

In [50]:
glob = initialize_policy_params_TS(standardize=False,baseline_features=[i for i in range(4)],psi_features=[],responsivity_keys=[i for i in range(4)])
               

In [36]:
users  = [int(np.random.random()*10) for i in range(100)]

In [10]:
#result = pyreadr.read_r('../../data/usertest/history.RData')
#result['data.history']

In [39]:
y = [np.random.normal(0,1.33) for i in range(100)]

In [40]:
def new_form(X):
    to_return = []
    for x in X:
        intercept = [1]
        base = intercept+x
        second = [.6*base[i] for i in range(len(base)) ]
        third = [int(np.random.random()>.5)-.6*base[i] for i in range(len(base)) ]
        to_return.append(base+second+third)
    return to_return
    
    

In [42]:
to_test_X = new_form(data)

In [44]:
#to_test_X 
#will probably need to standardize

In [53]:
temp_params = run_gpytorchkernel.run(np.array(to_test_X), users,np.array(y),glob)

In [ ]:
inv_term = simple_bandits.get_inv_term(temp_params['cov'],temp_data[0].shape[0],temp_params['noise'])
global_policy_params.update_params(temp_params)
global_policy_params.inv_term=inv_term

In [52]:
def get_posterior_parameters_all():
    pass